# Titanic Tree Modeling

In [1]:
# https://github.com/Swalloow/Kaggle/blob/master/Titanic%20Survivors/Titanic%20RandomForest.ipynb/
# 설명 : https://swalloow.github.io/decison-randomforest
import numpy as np
import pandas as pd

In [ ]:
# ◾Survived: 생존 여부 => 0 = No, 1 = Yes
# ◾pclass: 티켓 등급 => 1 = 1st, 2 = 2nd, 3 = 3rd
# ◾Sex: 성별
# ◾Age: 나이
# ◾Sibsp: 함께 탑승한 형제자매, 배우자의 수
# ◾Parch: 함께 탑승한 부모, 자식의 수
# ◾Ticket: 티켓 번호
# ◾Fare: 운임
# ◾Cabin: 객실 번호
# ◾Embarked: 탑승 항구 => C = Cherbourg, Q = Queenstown, S = Southampton

## Load Data

In [49]:
def Init_Data(titanic):
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 1
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 0
    
    titanic["Embarked"]=titanic["Embarked"].fillna("S")
    titanic.loc[titanic["Embarked"]=="S","Embarked"] = 0
    titanic.loc[titanic["Embarked"]=="C","Embarked"] = 1
    titanic.loc[titanic["Embarked"]=="Q","Embarked"] = 2

    return titanic

In [6]:
# titanic = pd.read_csv("../data/titan_train.csv")
# titanic = Init_Data(titanic)

import seaborn as sb
titanic_raw = sb.load_dataset('titanic')
titanic_raw.head()
titanic = titanic_raw
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [50]:
titanic = Init_Data(titanic)

In [51]:
titanic.dtypes
titanic.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'Sibsp', 'Parch', 'Fare',
       'Embarked', 'Class', 'Who', 'Adult_male', 'Deck', 'Embark_town',
       'Alive', 'Alone'],
      dtype='object')

In [52]:
titanic = titanic.rename(columns={'survived':'Survived', 'pclass':'Pclass', 'sex':'Sex', 'age':'Age', 'sibsp':'Sibsp', 'parch':'Parch', 'fare':'Fare',
       'embarked':'Embarked', 'class':'Class', 'who':'Who', 'adult_male':'Adult_male', 'deck':'Deck', 'embark_town':'Embark_town',
       'alive':'Alive', 'alone': 'Alone'
                   
                 })

In [53]:
#훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(titanic, test_size = 0.3)

train_set.shape
test_set.shape

(268, 15)

In [54]:
titanic_train = train_set
titanic_train.head()

,Survived,Pclass,Sex,Age,Sibsp,Parch,Fare,Embarked,Class,Who,Adult_male,Deck,Embark_town,Alive,Alone
100,0,3,0,28.0,0,0,7.8958,0,Third,woman,False,NaN,Southampton,no,True
51,0,3,1,21.0,0,0,7.8000,0,Third,man,True,NaN,Southampton,no,True
259,1,2,0,50.0,0,1,26.0000,0,Second,woman,False,NaN,Southampton,yes,False
855,1,3,0,18.0,0,1,9.3500,0,Third,woman,False,NaN,Southampton,yes,False
724,1,1,1,27.0,1,0,53.1000,0,First,man,True,E,Southampton,yes,False


In [55]:
titanic_test = test_set
titanic_test.head()

,Survived,Pclass,Sex,Age,Sibsp,Parch,Fare,Embarked,Class,Who,Adult_male,Deck,Embark_town,Alive,Alone
367,1,3,0,28.0,0,0,7.2292,1,Third,woman,False,NaN,Cherbourg,yes,True
159,0,3,1,28.0,8,2,69.5500,0,Third,man,True,NaN,Southampton,no,False
526,1,2,0,50.0,0,0,10.5000,0,Second,woman,False,NaN,Southampton,yes,True
648,0,3,1,28.0,0,0,7.5500,0,Third,man,True,NaN,Southampton,no,True
507,1,1,1,28.0,0,0,26.5500,0,First,man,True,NaN,Southampton,yes,True


## Data Preprocessing

In [48]:
# Preprocessing titanic dataset
def Init_Data(titanic):
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 1
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 0
    
    titanic["Embarked"]=titanic["Embarked"].fillna("S")
    titanic.loc[titanic["Embarked"]=="S","Embarked"] = 0
    titanic.loc[titanic["Embarked"]=="C","Embarked"] = 1
    titanic.loc[titanic["Embarked"]=="Q","Embarked"] = 2

    return titanic

In [56]:
# titanic_test = Init_Data(titanic_test)
# titanic_train = Init_Data(titanic_train)

In [57]:
predictors = ["Sex", "Age", "Embarked"]
x = titanic[predictors]
y = titanic["Survived"]

In [74]:
type(y)

pandas.core.series.Series

## DecisionTree

In [58]:
from sklearn.tree import DecisionTreeClassifier

In [59]:
decision = DecisionTreeClassifier().fit(x,y)
print(decision)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


## RandomForest

In [60]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
forst = RandomForestClassifier(n_jobs=-1).fit(x,y)
forst

C:\Jam\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

## Cross Validation

In [62]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [67]:
x

,Sex,Age,Embarked
0,1,22.0,0
1,0,38.0,1
2,0,26.0,0
3,0,35.0,0
4,1,35.0,0
5,1,28.0,2
6,1,54.0,0
7,1,2.0,0
8,0,27.0,0
9,0,14.0,1


In [69]:
y

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     0
15     1
16     0
17     1
18     0
19     1
20     0
21     1
22     1
23     1
24     0
25     1
26     0
27     0
28     1
29     0
      ..
861    0
862    1
863    0
864    0
865    1
866    1
867    0
868    0
869    1
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    1
881    0
882    0
883    0
884    0
885    0
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [63]:
decision_score = cross_val_score(decision, x, y, cv=5).mean()
forst_score = cross_val_score(forst, x, y, cv=5).mean()

print("DecisionTree = {0:.6f}".format(decision_score))
print("RandomForest = {0:.6f}".format(forst_score))

DecisionTree = 0.735092
RandomForest = 0.759824
